In [1]:
import os
import psutil
import numpy as np
import pandas as pd
import polars as pl
import warnings
warnings.filterwarnings("ignore")

from utils import *

In [2]:
# data경로 확보
full_path  = os.getcwd()
upper_path = os.path.dirname(full_path)
data_path  = os.path.join(upper_path, 'Data')

# csv파일 경로 확보
train_path  = os.path.join(data_path, 'train.csv')
test_path   = os.path.join(data_path, 'test.csv')
sample_path = os.path.join(data_path, 'sample_submission.csv')

In [3]:
# 데이터 읽기(polaris)
train  = pl.read_csv(train_path)
test   = pl.read_csv(test_path)
sample_submission = pl.read_csv(sample_path)

In [4]:
train

ID,Click,F01,F02,F03,F04,F05,F06,F07,F08,F09,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28,F29,F30,F31,F32,F33,F34,F35,F36,F37,F38,F39
str,i64,str,str,str,f64,str,i64,str,str,str,str,f64,str,str,i64,str,str,str,f64,f64,str,str,str,str,f64,str,str,f64,str,f64,str,str,f64,f64,str,str,f64,str,f64,str
"""TRAIN_00000000…",1,"""NSLHFNS""","""AVKQTCL""","""DTZFPRW""",114.0,"""ISVXFVA""",1,"""PQZBVMG""","""LPYPUNA""","""IZYJZDA""","""RANQNXO""",66.0,"""EGWPZEB""","""SMRBWMU""",4,"""NGMRRQG""","""NLHSWSR""","""SXZLOWA""",null,1.0,"""LTCDFSX""","""SWAZXZY""","""SNDDHSM""","""IDHAIQQ""",5.0,"""HLADEES""","""XAUNDQW""",3.0,"""MAVCFCM""",1.0,"""NZGEZLW""","""GTISJWW""",380.0,2.0,"""AXQFZWC""","""IRUDRFB""",null,"""TFJMLCZ""",0.0,"""AURZYDY"""
"""TRAIN_00000001…",0,"""VGIVWZQ""","""LSUSMVO""","""PQGWFJZ""",26.0,"""NFRVLWS""",43,"""IMPIGJT""","""MIGYEEG""","""NGODWIN""","""GBQMJYF""",137.0,"""OUPRLWN""","""QVLCQTS""",22,"""ZHOLRQX""","""DGLEHCI""","""ZSBOVQM""",19.0,1.0,"""QAOROGG""","""BUUUKVW""","""SNDDHSM""","""IDHAIQQ""",21.0,"""TPNEYOY""","""BUWDIBR""",32.0,"""IYZTLFQ""",2.0,"""NZGEZLW""","""GTISJWW""",466.0,1.0,"""DRVVDHZ""","""IRUDRFB""",19.0,"""AUGTURV""",0.0,"""LUZRMLU"""
"""TRAIN_00000002…",0,"""JCDXFYU""","""PILDDJU""","""IAGJDOH""",119.0,"""LFPUEOV""",0,"""FFUTIRZ""","""OFKQGTY""","""BEZTQIO""","""YLKUVQA""",null,"""MGRUYII""","""KGJACUM""",0,"""ZNLREKK""","""PBCDAMR""","""TEHYADQ""",2.0,0.0,"""TGVLSYT""","""FUZMWEL""","""SNDDHSM""","""ZXHXBZQ""",null,"""YMGUAHK""","""WVRHGBC""",null,"""VNBXRYV""",null,"""VHXETCF""","""KHZNEZF""",197.0,0.0,"""QMOULXS""","""IRUDRFB""",8.0,"""ZVSTLNM""",0.0,"""MHBRSQK"""
"""TRAIN_00000003…",1,"""PSMFWTP""","""ZYAVJHP""",null,15.0,"""ATQPZSJ""",26,"""ZDTZNSB""","""THBWWCD""","""LTETYBG""","""GEKHGQZ""",50.0,"""OPGPVBH""","""UMIEGWH""",20,null,"""WDTKLCQ""","""SXZLOWA""",14.0,0.0,null,"""BJAYVHN""","""SNDDHSM""","""VTYMMMX""",87.0,"""IROGIOW""",null,2.0,"""YQCIJMD""",1.0,"""IVIRTPR""","""GTISJWW""",8640.0,0.0,"""IZLJUJS""","""IRUDRFB""",14.0,"""ZBSRLCQ""",0.0,"""GAZBSSZ"""
"""TRAIN_00000004…",0,"""SLCRICD""","""QPQWGXA""",null,13.0,"""CHZGJZR""",20,"""PQZBVMG""","""MIGYEEG""","""LJBQPJW""","""SOKLCDW""",789.0,"""LHVQODU""","""YJFFIGH""",42,null,"""NRRIGTN""","""TEHYADQ""",13.0,0.0,null,"""LBABTIR""","""NXBZHKD""","""IZDGSXJ""",null,"""PWGXQTB""",null,null,"""IYZTLFQ""",null,"""NZGEZLW""","""WHSRKIM""",41774.0,0.0,"""BHBIZCL""","""IRUDRFB""",13.0,"""QHYLSBX""",0.0,"""QTATWAY"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""TRAIN_28605386…",0,"""NLVEHEJ""","""TQLUNHQ""","""DTZFPRW""",7.0,"""TSAHWRE""",1,"""RLRNZDM""","""SODVHOK""","""IZYJZDA""","""ZPMODDZ""",213.0,"""VRGCGWA""","""SMRBWMU""",12,"""NGMRRQG""","""FBGVORR""","""FMJARXS""",6.0,2.0,"""OYRTDCM""","""SWAZXZY""","""SNDDHSM""","""IDHAIQQ""",6.0,"""ELRLVHO""","""HJHEDHR""",62.0,"""RSJLBWR""",5.0,"""TANUHHL""","""GTISJWW""",43.0,3.0,"""PNSISZF""","""IRUDRFB""",6.0,"""TFJMLCZ""",0.0,"""ZBOTIHF"""
"""TRAIN_28605387…",0,"""GGCUFPH""","""JLGELTR""","""IAGJDOH""",null,"""MZOKJAS""",26,"""GIHHVOP""","""IGQGIJM""","""KLFHAUF""","""HBFALWV""",47.0,"""LFZZAOB""","""BFVIKOK""",7,"""NGMRRQG""","""XVTICSY""","""ORHLUGM""",null,0.0,"""JLITBKP""","""YBGYGNW""","""SNDDHSM""","""IDHAIQQ""",null,"""OZQAFLN""","""WVRHGBC""",12.0,"""RSJLBWR""",3.0,"""FGOVFJM""","""GTISJWW""",4341.0,0.0,"""AJHSFUP""","""IRUDRFB""",null,"""BBDNLVH""",0.0,"""LXTGFRC"""
"""TRAIN_28605388…",0,"""SNXDPNU""","""VTPZTHN""","""KFOPSGX""",92.0,"""ZTAHYDS""",3,"""JXOLWIU""","""CZDUWVM""","""QBSGNGN""","""EWFXKQY""",null,"""IYCTPAY""","""FGQOGSD""",21,"""ZHOLRQX""","""EDMHVZZ""","""SXZLOWA""",20.0,1.0,"""UDVKJTH""","""SWAZXZY""","""UHZOZZL""","""IDHAIQQ""",59.0,"""QIWQMAW""","""YYQVFBZ""",null,"""YYEFTXW""",null,"""SLXYBBG""","""WCPHYPZ""",25.0,2.0,"""XIPYLHN""","""IRUDRFB""",20.0,"""OOQBJMI""",0.0,"""XIWMBGB"""


In [36]:
target = "Click"

In [53]:
dfs = [train,test]
# 각 열을 순회하며 결측치의 비율 계산
for df in dfs:
    print(df)
    for col in df.columns:
        # 결측치 비율 계산
        missing_percentage = df[col].is_null().mean()
        print(f"{col}: {missing_percentage*100:.2f}% missing")
    print('_____________')

shape: (28_605_391, 41)
┌────────────────┬───────┬─────────┬─────────┬───┬──────┬─────────┬─────┬─────────┐
│ ID             ┆ Click ┆ F01     ┆ F02     ┆ … ┆ F36  ┆ F37     ┆ F38 ┆ F39     │
│ ---            ┆ ---   ┆ ---     ┆ ---     ┆   ┆ ---  ┆ ---     ┆ --- ┆ ---     │
│ str            ┆ i64   ┆ str     ┆ str     ┆   ┆ f64  ┆ str     ┆ f64 ┆ str     │
╞════════════════╪═══════╪═════════╪═════════╪═══╪══════╪═════════╪═════╪═════════╡
│ TRAIN_00000000 ┆ 1     ┆ NSLHFNS ┆ AVKQTCL ┆ … ┆ null ┆ TFJMLCZ ┆ 0.0 ┆ AURZYDY │
│ TRAIN_00000001 ┆ 0     ┆ VGIVWZQ ┆ LSUSMVO ┆ … ┆ 19.0 ┆ AUGTURV ┆ 0.0 ┆ LUZRMLU │
│ TRAIN_00000002 ┆ 0     ┆ JCDXFYU ┆ PILDDJU ┆ … ┆ 8.0  ┆ ZVSTLNM ┆ 0.0 ┆ MHBRSQK │
│ TRAIN_00000003 ┆ 1     ┆ PSMFWTP ┆ ZYAVJHP ┆ … ┆ 14.0 ┆ ZBSRLCQ ┆ 0.0 ┆ GAZBSSZ │
│ TRAIN_00000004 ┆ 0     ┆ SLCRICD ┆ QPQWGXA ┆ … ┆ 13.0 ┆ QHYLSBX ┆ 0.0 ┆ QTATWAY │
│ …              ┆ …     ┆ …       ┆ …       ┆ … ┆ …    ┆ …       ┆ …   ┆ …       │
│ TRAIN_28605386 ┆ 0     ┆ NLVEHEJ ┆ TQLUNHQ ┆ … ┆ 6

In [5]:
def filter_cols(df): #결측치가 70%이상 되는 피처들 제외
    for col in df.columns:
        if col not in ["Click", "ID"]:
            isnull = df[col].is_null().mean()
            if isnull > 0.35:
                df = df.drop(col)
    return df 

In [6]:
train1 = filter_cols(train)
test1 = filter_cols(test)

In [7]:
print(train1.columns)
print(test1.columns)

['ID', 'Click', 'F01', 'F02', 'F04', 'F05', 'F06', 'F07', 'F08', 'F09', 'F10', 'F11', 'F12', 'F13', 'F14', 'F16', 'F17', 'F18', 'F19', 'F21', 'F22', 'F23', 'F24', 'F25', 'F28', 'F30', 'F31', 'F32', 'F33', 'F34', 'F35', 'F36', 'F37', 'F38', 'F39']
['ID', 'F01', 'F02', 'F04', 'F05', 'F06', 'F07', 'F08', 'F09', 'F10', 'F11', 'F12', 'F13', 'F14', 'F16', 'F17', 'F18', 'F19', 'F21', 'F22', 'F23', 'F24', 'F25', 'F28', 'F30', 'F31', 'F32', 'F33', 'F34', 'F35', 'F36', 'F37', 'F38', 'F39']


In [ ]:
train1  = train1.to_pandas()
test1 = test1.to_pandas()

: 

In [ ]:
train1.to_csv('/home/sgh/yes/envs/DACON/(3rd)Click_Ratio/Data/train1', index=False)
test1.to_csv('/home/sgh/yes/envs/DACON/(3rd)Click_Ratio/Data/test1', index=False)